In [ ]:
!apt -qq update
!apt -qq -y install libnvtoolsext1
!apt -qq -y install xvfb freeglut3-dev ffmpeg
!pip -q install gym
!pip -q install pyglet
!pip -q install pyopengl
!pip -q install pyvirtualdisplay
!pip install -q gym-super-mario-bros==7.3.0
!pip install -q opencv-python


8 packages can be upgraded. Run 'apt list --upgradable' to see them.
libnvtoolsext1 is already the newest version (11.5.114~11.5.1-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  freeglut3 libegl-dev libfontenc1 libgl-dev libgl1-mesa-dev libgles-dev
  libgles1 libglu1-mesa libglu1-mesa-dev libglvnd-core-dev libglvnd-dev
  libglx-dev libice-dev libopengl-dev libsm-dev libxfont2 libxkbfile1
  libxt-dev x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common
Suggested packages:
  libice-doc libsm-doc libxt-doc
The following NEW packages will be installed:
  freeglut3 freeglut3-dev libegl-dev libfontenc1 libgl-dev libgl1-mesa-dev
  libgles-dev libgles1 libglu1-mesa libglu1-mesa-dev libglvnd-core-dev
  libglvnd-dev libglx-dev libice-dev libopengl-dev libsm-dev libxfont2
  libxkbfile1 libxt-dev x11-xkb-utils xfonts-base xfonts-enco

In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1024, 768))
display.start()
import os
#os.environ["DISPLAY"] = ":" + str(display.display) + "." + str(display.screen)

import torch
from torch import nn
from torchvision import transforms as T
from PIL import Image
import numpy as np
from pathlib import Path
from collections import deque
import random, datetime, os, copy
from enum import IntEnum
import numpy
import matplotlib.pyplot as plt
import sys
import cv2
from google.colab.patches import cv2_imshow

# Gym is an OpenAI toolkit for RL
import gym
from gym.spaces import Box
from gym.wrappers import FrameStack

# NES Emulator for OpenAI Gym
from nes_py.wrappers import JoypadSpace




In [ ]:
def create_gene(INDIVIDUALS,MAX_TIME):
    generation = []
    for x in range(INDIVIDUALS):
        gene = []
        for y in range(MAX_TIME):
            gene.append(random.choice([0, 1]))
        generation.append(gene)

    return generation

#2点交叉
def two_points_cross(genes1,genes2):
    max_index = min(len(genes1), len(genes2))
    index1 = numpy.random.randint(0, max_index)
    index2 = numpy.random.randint(index1, max_index)
    indices = (index1, index2)
    c_genes1=[]
    c_genes2=[]
    c_genes1 = numpy.hstack((genes1[:indices[0]],genes2[indices[0]:indices[1]],genes1[indices[1]:]))
    c_genes2 = numpy.hstack((genes2[:indices[0]],genes1[indices[0]:indices[1]],genes2[indices[1]:]))
    return c_genes1,c_genes2

# def one_point_cross(gene1, gene2):  #1点交叉 #使わなかった
#     cross_point = random.choice(range(len(gene1)))
#     new_gene1 = gene1[:cross_point] + gene2[cross_point:]
#     new_gene2 = gene2[:cross_point] + gene1[cross_point:]
#     return new_gene1, new_gene2

#突然変異用の関数
def mutation(genes1,genes2,mutation_rate):

    # 子の遺伝子情報
    c_genes1 = []
    c_genes2 = []
    for i in range(len(genes1)):  # 各遺伝子を走査
        # 突然変異
        if random.random() < mutation_rate:
            if genes1[i]==1:
              genes1[i]=0
            else:
              genes1[i]=1
        if random.random() < mutation_rate:
            if genes2[i]==1:
              genes2[i]=0
            else:
              genes2[i]=1

        c_genes1.append(genes1[i])
        c_genes2.append(genes2[i])

    return c_genes1,c_genes2

#遺伝子をscore順にソートする関数
def sorts(genes, scores):
    sorted_data = sorted(zip(scores,genes), reverse=True)
    sorted_scores,sorted_GENES = zip(*sorted_data)
    return list(sorted_GENES), list(sorted_scores)




/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Super Mario environment for OpenAI Gym
import gym_super_mario_bros


# Initialize Super Mario environment
# https://github.com/Kautenja/gym-super-mario-bros
env = gym_super_mario_bros.make("SuperMarioBros-1-1-v0")

# Limit the action-space to
#   0. walk right
#   1. jump right
env = JoypadSpace(env, [["right","B"], ["right", "A","B"]]) #早くクリアするために右ダッシュと右ダッシュジャンプのみ可能とした


env.reset()

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment SuperMarioBros-1-1-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


array([[[104, 136, 252],
        [104, 136, 252],
        [104, 136, 252],
        ...,
        [104, 136, 252],
        [104, 136, 252],
        [104, 136, 252]],

       [[104, 136, 252],
        [104, 136, 252],
        [104, 136, 252],
        ...,
        [104, 136, 252],
        [104, 136, 252],
        [104, 136, 252]],

       [[104, 136, 252],
        [104, 136, 252],
        [104, 136, 252],
        ...,
        [104, 136, 252],
        [104, 136, 252],
        [104, 136, 252]],

       ...,

       [[240, 208, 176],
        [228,  92,  16],
        [228,  92,  16],
        ...,
        [228,  92,  16],
        [228,  92,  16],
        [  0,   0,   0]],

       [[240, 208, 176],
        [228,  92,  16],
        [228,  92,  16],
        ...,
        [228,  92,  16],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[228,  92,  16],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [228,  92,  16]]

In [ ]:
from gym.wrappers import GrayScaleObservation
import cv2
from matplotlib import pyplot as plt
env = GrayScaleObservation(env , keep_dim=True)

In [ ]:
state, reward, done, info = env.step(action=0)
init_state= state.reshape([240,256]) #3D to 2D
unique_arr = np.unique(init_state)   #Instances
template_path="/content/turtle1.png"
jump_action=0


def enemy_each_block(target_are, target_list):
  tmp2 = np.full_like(target_are, False, dtype=bool)
  #target_are[0][0]=5
  #print(target_are)
  for number in target_list:
    tmp3 = np.where(target_are == number, True, False)
    tmp2 = tmp2 | tmp3

  return(np.all(tmp2))

def enemy(tmp,unique_arr):
  # tmp = in_state.reshape([240,256]) #3D to 2D
  # #unique_arr = np.unique(tmp)   #Instances
  #unique_arr= np.delete(unique_arr, np.where(unique_arr == 124) , None)
  tmp=tmp
  #Ground starts from 209 and Ground color=124
  for i in range(240):
    if( i >= 209):
      tmp[i]=0

  #plt.imshow(tmp)
  #Mario=107 Mario's size=15
  #Get Mario's position
  search_number = 107
  mario_width=15
  out = [False]
  pos = np.where(tmp == 107)  #Extract Mario's position
  if pos[0].size > 0:
    y_min=np.min(pos[0])
    y_max=np.max(pos[0])
    x_min=np.min(pos[1])
    x_max=np.max(pos[1])
    mario=tmp[y_min:y_max,x_min:x_max] #Mario
    mario_r= tmp[y_min:y_max,x_max+1:x_max+1+mario_width] #Mario right
    mario_l= tmp[y_min:y_max,x_min-1:x_max-1-mario_width] #Mario left
    mario_t= tmp[y_min-1:y_min-1-mario_width,x_min:x_max] #Mario top
    mario_tr= tmp[y_min-1:y_min-1-mario_width, x_max+1:x_max+1+mario_width] #Mario top-right
    mario_tl= tmp[y_min-1:y_min-1-mario_width, x_min-1:x_max-1-mario_width] #Mario top-left
    mario_d= tmp[y_max+1:y_max+1+mario_width, x_min:x_max] #Mario down
    mario_dr= tmp[y_max+1:y_max+1+mario_width, x_max+1:x_max+1+mario_width]#Mario down-right
    mario_dl= tmp[y_max+1:y_max+1+mario_width, x_min-1:x_max-1-mario_width]#Mario down-left

    #enemy_block=[mario_tl, mario_t, mario_tr, mario_r, mario_dr, mario_d, mario_dl, mario_l]
    enemy_block=[mario_r]
    #for tmp_block in enemy_block:
    out=enemy_each_block(mario_r, unique_arr)
  return out

def avoid_mushroom(move,state,frame):
    global jump_action
    tmp = state.reshape([240,256]) #3D to 2D
    unique_arr = np.unique(tmp)   #Instances
    unique_arr= np.delete(unique_arr, np.where(unique_arr == 124) , None)
    out=enemy(tmp,unique_arr)

    #Ground starts from 209 and Ground color=124
    for i in range(240):
      if( i >= 209):
        tmp[i]=0

    # テンプレート画像読み込み
    template = cv2.imread(template_path)
    template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

    action=move
    pos = np.where(tmp == 107)  #Extract Mario's position
    ret_image = frame.copy()
    if out:
      # グレースケール変換
      img_gray = cv2.cvtColor(ret_image, cv2.COLOR_BGR2GRAY)
      for i in range(90,100,5):
        temp= template_gray.copy()
        temp1 = cv2.resize(temp, None, fx=i/100, fy=i/100, interpolation=cv2.INTER_CUBIC)
        h,w = temp1.shape
        # テンプレートマッチング
        result = cv2.matchTemplate(img_gray, temp1, cv2.TM_CCOEFF_NORMED)
        loc = np.where(result > 0.9)

      # マッチング結果の処理
        for pt in zip(*loc[::-1]):
            ret_image = cv2.rectangle(ret_image, pt, (pt[0] + w, pt[1] + h), (0, 0, 255), 2)

      # マッシュルームを回避するアクション選択
        if len(loc[0]) > 0:
            # マッシュルームが検出された場合は→+ジャンプで移動
            action = 1
            break

            #cv2_imshow(ret_image)

    if action==1:
      jump_action+=1
    if jump_action>16:
      action=0
      jump_action=0
    return action

def mario_surroundings(state):
    tmp = state.reshape([240,256]) #3D to 2D
    for i in range(240):
        if( i >= 209):
          tmp[i]=0
    search_number = 107
    mario_width=15
    mario_surroundings=[]
    pos= np.where(tmp == 107)  #Extract Mario's position
    if pos[0].size > 0:
      surrounding_size = 20  # The size of the surroundings to consider. Adjust as needed.
      y_min=min(0,np.min(pos[0])-surrounding_size)
      y_max=max(tmp.shape[0],np.max(pos[0])+surrounding_size)
      x_min=min(0,np.min(pos[1])-surrounding_size)
      x_max=max(tmp.shape[1],np.max(pos[1])+surrounding_size)
      mario_r= tmp[y_min:y_max,x_max+1:x_max+1+mario_width] #Mario right
      mario_surroundings = tmp[mario_r]

    return mario_surroundings


def detect_mushrooms(gen,state):
    # Initialize an empty list to store the locations of detected mushrooms
    new_gen=gen
    mushroom_template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
    threshold=0.1
    screen=mario_surroundings(state)
    # for k in range (len(screen)):
    # Iterate over each pixel in the screen

    for i in range(screen.shape[0]):
        breaker=False
        for j in range(screen.shape[1]):
            # Extract a patch from the screen at the current location
            patch = screen[i:i+mushroom_template.shape[0], j:j+mushroom_template.shape[1]]

            # If the patch size is not the same as the mushroom template, skip this location
            if patch.shape != mushroom_template.shape:
                continue

            # Calculate the difference between the patch and the mushroom template
            difference = np.sum((patch - mushroom_template)**2)

            # If the difference is below a certain threshold, consider this location as a mushroom
            if difference < threshold:
                if random.random()<0.8:
                  new_gen=1
                breaker=True
                break
        if breaker:
            break
    # if breaker:
    #     break

    return new_gen


out=enemy(init_state,unique_arr)
print(out)
print(unique_arr)



True
[  0  99 107 124 140 157 177 203 214 252]


/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


In [ ]:
# 試行回数
EPISODE_NUMB = 1

# 遺伝子長
MAX_TIME = 8000 #STEP_COUNTを使用する場合には800

#生成個体数
INDIVIDUALS=20

#遺伝回数
TRY_GENERATION=300 #30ぐらいなら可能性あり(two_point)

#1アクションで行う行動回数
STEP_COUNT=10


In [ ]:
import copy
import sys
import numpy
import time

#https://qiita.com/nsd24/items/7758410128872d684e05
#GA:https://morishin.hatenablog.com/entry/marioai-2009

Max_generation=[]
GENES = create_gene(INDIVIDUALS,MAX_TIME)
for x in range(TRY_GENERATION):
    generation_scores = []
    print("TRY_GENERATION",x)
    for i, gen in enumerate(GENES):
        Goal=False;
        breaker = False
        state = env.reset() # 環境リセット
        now_x_position=0
        m_cnt_super_not_change=0
        for j,move in enumerate(gen):
          #for i in range(STEP_COUNT):
            state, reward, done, info = env.step(move)
            frame=copy.deepcopy(env.render(mode='rgb_array'))

            if j!=len(gen)-1:
              gen[j+1]=avoid_mushroom(gen[j+1],state,frame)
              #gen[j+1]=avoid_mushroom(move,state,frame)

            if now_x_position==info["x_pos"]:
              m_cnt_super_not_change+=1
            else:
              now_x_position=info["x_pos"]
              m_cnt_super_not_change=0
            if m_cnt_super_not_change>3000:
              break

            if info['flag_get']:    #ゴール判定
                print(f"<<< Mario get the flag. GOOOOOOOOOOOOOOOOOOOOOAL! >>>")
                Goal=True
                breaker=True
                break

            if done:
                breaker=True              #落下した場合その遺伝子によるプレイは終了
                break
          # if breaker:
          #   break
        generation_scores.append(info["x_pos"])
        print("generation: {} genes: {} stops: {} score: {}".format(x + 1, i + 1, j + 1, info["x_pos"]))
        if Goal:
            break


    GENES, generation_scores=sorts(GENES, generation_scores)
    print(generation_scores[0])
    Max_generation.append(generation_scores[0])  #x世代の最大スコアをリストに追加
    if Goal:
      print(GENES[0])
      f = open('myfile.txt', 'w')
      f.writelines(str(GENES[0]))
      f.close()
      break


    NEW_GENES = []
    for k in range(9):
        gene1, gene2 = random.sample(list(GENES[:10]), 2)

        gene1, gene2 = two_points_cross(gene1, gene2)
        gene1, gene2 = mutation(gene1, gene2,0.2)
        NEW_GENES.append(gene1)
        NEW_GENES.append(gene2)
    NEW_GENES = GENES[:2] + NEW_GENES
    GENES = NEW_GENES




TRY_GENERATION 0


/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:280: UserWarning: WARN: No render modes was declared in the environment (env.metadata['render_modes'] is None or not defined), you may have trouble when calling `.render()`.
  logger.warn(


error: ignored

In [ ]:
frames=[]

#世代ごとの最大スコアの推移をグラフ化
plt.plot(Max_generation)
plt.xlabel('世代数')
plt.ylabel('Awards')
plt.title('MarioAI by GA')
plt.show()

print(GENES)
for i in range(EPISODE_NUMB):
    observation = env.reset()  # reset for each new trial
    done = False
    total_reward = 0
    total_time = 0
    time = 0

    print(generation_scores[:1])

    for i, gen in enumerate(GENES[:1]):
        state = env.reset() # 環境リセット
        breaker = False
        for j,move in enumerate(gen):
            state, reward, done, info = env.step(move)
            frames.append(copy.deepcopy(env.render(mode='rgb_array')))
            total_reward += reward


            if done:
                breaker = True
                break
        if breaker:
            break
    print('test episode:', i, 'reward:', total_reward, 'time:', j)


env.close()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation
import numpy as np
from IPython.display import HTML

import matplotlib.pyplot as plt
import matplotlib.animation
import numpy as np
from IPython.display import HTML
from matplotlib.animation import PillowWriter

# アニメーションのサイズと解像度を調整
fig, ax = plt.subplots(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi=72)
ax.axis('off')

# imshow()を使って描画
patch = ax.imshow(frames[0])

# アニメーションの更新関数
def animate(i):
    patch.set_data(frames[i])

# FuncAnimationを作成
ani = matplotlib.animation.FuncAnimation(fig, animate, frames=len(frames), interval=50)

# アニメーションをGIFファイルとして保存
writer = PillowWriter(fps=30)
ani.save('animation.gif', writer=writer)

# アニメーションを表示
HTML(ani.to_jshtml())



# plt.figure(figsize=(frames[0].shape[1] / 36.0, frames[0].shape[0] / 36.0), dpi = 36)
# patch = plt.imshow(frames[0])
# plt.axis('off')
# animate = lambda i: patch.set_data(frames[i])
# ani = matplotlib.animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval = 50)
# HTML(ani.to_jshtml())

In [ ]:
import numpy as np
SKIP_RATE = 2
if SKIP_RATE == 1:
  frames_new = np.array(frames)
else:
  np_frames = np.array(frames)
  frames_new = np_frames[::SKIP_RATE].tolist()
matplotlib.rcParams['animation.embed_limit'] = 2**128
plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
patch = plt.imshow(frames_new[0])
plt.axis('off')
animate = lambda i: patch.set_data(frames_new[i])
ani = matplotlib.animation.FuncAnimation(plt.gcf(), animate, frames=len(frames_new), interval = 50)
HTML(ani.to_jshtml())

In [ ]:
# Reference
# https://stable-baselines3.readthedocs.io/en/master/guide/examples.html#bonus-make-a-gif-of-a-trained-agent
import imageio
imageio.mimsave('ai_mario.gif', [np.array(img) for i, img in enumerate(frames) if i%SKIP_RATE == 0], fps=29)

In [ ]:
from google.colab import files
# files.download('ai_mario.gif')
